In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt  

train_path=r'D:\LJK\pingan\PINGAN-2018-train_demo.csv'
train=pd.read_csv(train_path)
train.head()

train['TIME_Tsfd']=0

train['TIME_Tsfd']=train['TIME'].apply(lambda x:datetime.datetime.fromtimestamp(x))

train['TIME_Tsfd']=pd.to_datetime(train['TIME_Tsfd'])
train['Month']=train['TIME_Tsfd'].dt.month
train['hour']=train['TIME_Tsfd'].dt.hour
train['week_of_year']=train['TIME_Tsfd'].dt.weekofyear
train['day_of_week']=train['TIME_Tsfd'].dt.dayofweek

train.head()

#计算每个用户，每个trip_id的平均速度
train_speed_grouped=train.groupby(['TERMINALNO','TRIP_ID'])['SPEED'].agg(['mean','max'])
train_speed_grouped.reset_index(inplace=True)
train_speed_grouped_1=train_speed_grouped.groupby('TERMINALNO')['mean'].agg(['mean','max'])
train_speed_grouped_2=train_speed_grouped.groupby('TERMINALNO')['max'].agg(['mean','max'])
train_speed_grouped_1.reset_index(inplace=True)
train_speed_grouped_2.reset_index(inplace=True)
train_speed_grouped_3=train.groupby('TERMINALNO')['Y'].agg(['mean'])
train_speed_grouped_3.reset_index(inplace=True)
train_speed_grouped_3.rename(columns={'mean':'Y'},inplace=True)

temp=pd.merge(train_speed_grouped_1,train_speed_grouped_2,on='TERMINALNO',how='left')
temp=pd.merge(temp,train_speed_grouped_3,on='TERMINALNO',how='left')
temp.rename(columns={'mean_x':'speed_tripmean_mean','mean_y':'speed_tripmax_mean',
                    'max_x':'speed_tripmean_max','max_y':'speed_tripmax_max'},inplace=True)
#统计每个用户的trip数
train_trip_numed=train.groupby(['TERMINALNO'])['TRIP_ID'].agg(['max'])
train_trip_numed.reset_index(inplace=True)
train_trip_numed.rename(columns={'max':'trip_num'},inplace=True)
temp=pd.merge(temp,train_trip_numed,on='TERMINALNO',how='left')
                 
print(temp.head())
plt.figure(1)  
plt.scatter(temp['speed_tripmean_mean'],temp['Y'], marker = 'x', color = 'm', label='1', s = 30)
plt.figure(2)  
plt.scatter(temp['speed_tripmean_max'],temp['Y'], marker = 'x', color = 'm', label='1', s = 30)
plt.figure(3)  
plt.scatter(temp['speed_tripmax_mean'],temp['Y'], marker = 'x', color = 'm', label='1', s = 30)
plt.figure(4)  
plt.scatter(temp['speed_tripmax_max'],temp['Y'], marker = 'x', color = 'm', label='1', s = 30)
plt.figure(5)  
plt.scatter(temp['trip_num'],temp['Y'], marker = 'x', color = 'm', label='1', s = 30)
plt.show()



   TERMINALNO  speed_tripmean_mean  speed_tripmean_max  speed_tripmax_mean  \
0           1            14.819857           32.476667           18.645111   
1           2             3.850738           26.009286            6.780909   
2           3             7.161065           13.425942           15.521351   
3           4             6.176600           16.111333           10.970918   
4           5             6.401703           32.544444           11.476118   

   speed_tripmax_max    Y  trip_num  
0          32.779999  0.0        45  
1          36.119999  0.0        66  
2          25.440001  0.0        37  
3          33.310001  0.0        98  
4          53.480000  0.0        85  


In [9]:
train_hour_num=train.groupby(['TERMINALNO','hour'])['TRIP_ID'].agg(['count'])
train_hour_num.reset_index(inplace=True)
train_trip_sum=train.groupby('TERMINALNO')['TRIP_ID'].agg(['count'])
train_trip_sum.reset_index(inplace=True)
train_hour_num=pd.merge(train_hour_num,train_trip_sum,on='TERMINALNO',how='left')
train_hour_num['count_ratio']=train_hour_num['count_x']/train_hour_num['count_y']


for i in range(0,24):
    h='h'+str(i)
    temp[h]=0
    train_hour_num[i]=0
str(train_hour_num['hour'])
#train_hour_num[str(train_hour_num['hour'])]
#for i in train['TERMINALNO'].unique():
 #   train_hour_num[str(train_hour_num['hour'])]=train_hour_num['count_ratio'].loc[train['TERMINALNO']==i]
#for i in train['TERMINALNO'].nunique():
#for i in range(len(train_hour_num['TERMINALNO'])):
  #  train_hour_num[train_hour_num['hour']][i]=train_hour_num['count_ratio'][i]
#train_hour_num

#for i in temp['TERMINALNO']:
  #  for j in range(0,24):
   #     h='h'+str(j)
    #    temp[h].loc[train_hour_num['TERMINALNO']==i]=train_hour_num['count_ratio'].loc[(train_hour_num['TERMINALNO']==i)&(train_hour_num['hour']==j)]


'0        0\n1        1\n2        8\n3       13\n4       16\n5       17\n6       18\n7       20\n8       21\n9       22\n10      23\n11       0\n12       1\n13      11\n14      12\n15      13\n16      14\n17      15\n18      16\n19      17\n20      18\n21      20\n22      21\n23       7\n24       8\n25       9\n26      10\n27      11\n28      17\n29      18\n        ..\n1288     8\n1289     9\n1290    10\n1291    11\n1292    15\n1293    16\n1294    17\n1295    18\n1296    19\n1297     9\n1298    12\n1299    13\n1300    14\n1301    15\n1302    16\n1303    17\n1304    18\n1305    20\n1306    21\n1307     7\n1308     8\n1309     9\n1310    10\n1311    11\n1312    12\n1313    13\n1314    14\n1315    15\n1316    16\n1317    17\nName: hour, Length: 1318, dtype: int64'

In [3]:
1+1

2